# Main analysis script

Put your leakpro results in the *data* folder using the following structure

data/main/*dataset*/*target_model*/.../result.json

Example:

data/main/ELD/LSTM/LiRA-num_shadow_model=64-signal_name=RescaledSMAPELoss.../result.json


In [1]:
from utils import objects_to_ablations
import glob, os, json, gc

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
data_folder = "data/main"
output_folder = "output/main"

# This is the variable that varies and we want to observe the effect of
important_name = "signal_name"

# This is the parameters given in the structure of the data
from_path = ["dataset", "target_model"]

# The parameters given in config in result.json
from_config = ["online", "audit", "signal_name"]

def title_format(parameters):
    s = f'{parameters["dataset"]} ({parameters["target_model"]}) - {parameters["attack"]}'
    if "online" in parameters.keys():
        if parameters["online"]:
            s = s + " online"
        else:
            s = s + " offline"
    if "audit" in parameters.keys():
        if parameters["audit"]:
            s = s + " audit"
        else:
            s = s + " attack"
    s = s + " ROC Curve"
    return s

config = {
    "ds_indivs": {
        "EEG": 32,
        "ELD": 32,
    },
    "indiv_strategy": "indiv_mle", # indiv_mean, indiv_median, indiv_outlier, indiv_mle
    "order": ["MSELoss", "MAELoss", "SMAPELoss", "RescaledSMAPELoss", "TrendLoss", "SeasonalityLoss", "TS2VecLoss"],
    "title_format": title_format,
}


In [ ]:
objects = []

for path in glob.glob(data_folder + "/**/*.json", recursive=True):
    # Add parameters from path
    data_folder_split = os.path.join(data_folder).split(os.sep)
    path_split = path.split(os.sep)[len(data_folder_split):]
    parameters = {param: path_split[i] for i, param in enumerate(from_path)}

    # Load json file
    with open(path) as f:
        data = json.load(f)

    # Add parameters from config in result.json
    parameters |= {k: v for k,v in data["config"].items() if k in from_config}
    if "signal_names" in data["config"].keys() and "signal_name" in from_config:
        parameters["signal_name"] = ",".join(data["config"]["signal_names"])

    # Assume attack name to be part of parameters
    parameters |= {"attack": data["result_name"]}
    
    if important_name not in parameters.keys():
        parameters[important_name] = parameters["attack"]

    important_value = parameters[important_name]
    parameters = {k: v for k,v in parameters.items() if k!=important_name}

    objects.append((data, parameters, important_value))
    print((parameters, important_value))


({'dataset': 'EEG', 'target_model': 'NHiTS', 'online': False, 'attack': 'DTS-MIA'}, 'DTS-MIA')
({'dataset': 'EEG', 'target_model': 'NHiTS', 'online': True, 'attack': 'DTS-MIA'}, 'DTS-MIA')
({'dataset': 'EEG', 'target_model': 'NHiTS', 'online': False, 'attack': 'LiRA'}, 'MAELoss')
({'dataset': 'EEG', 'target_model': 'NHiTS', 'online': False, 'attack': 'LiRA'}, 'MSELoss')
({'dataset': 'EEG', 'target_model': 'NHiTS', 'online': False, 'attack': 'LiRA'}, 'RescaledSMAPELoss')
({'dataset': 'EEG', 'target_model': 'NHiTS', 'online': False, 'attack': 'LiRA'}, 'SeasonalityLoss')
({'dataset': 'EEG', 'target_model': 'NHiTS', 'online': False, 'attack': 'LiRA'}, 'SMAPELoss')
({'dataset': 'EEG', 'target_model': 'NHiTS', 'online': False, 'attack': 'LiRA'}, 'TrendLoss')
({'dataset': 'EEG', 'target_model': 'NHiTS', 'online': False, 'attack': 'LiRA'}, 'TS2VecLoss')
({'dataset': 'EEG', 'target_model': 'NHiTS', 'online': True, 'attack': 'LiRA'}, 'MAELoss')
({'dataset': 'EEG', 'target_model': 'NHiTS', 'onlin

In [4]:
ablations = objects_to_ablations(objects, config)
for study in ablations:
    save_dir = "-".join(f"{k}={v}" for k, v in study.parameters.items())
    save_dir = os.path.join(output_folder, save_dir)
    os.makedirs(save_dir, exist_ok=True) 
    study.make_roc_plot(save_dir)
    study.make_table(save_dir)
    study.make_indiv_roc_plot(save_dir)
    gc.collect() # Garbage collect
    print(f"Saved study to {save_dir}")

Saved study to output/main/dataset=EEG-target_model=NHiTS-online=False-attack=DTS-MIA
Saved study to output/main/dataset=EEG-target_model=NHiTS-online=True-attack=DTS-MIA
Saved study to output/main/dataset=EEG-target_model=NHiTS-online=False-attack=LiRA
Saved study to output/main/dataset=EEG-target_model=NHiTS-online=True-attack=LiRA
Saved study to output/main/dataset=EEG-target_model=NHiTS-online=False-attack=RMIA
Saved study to output/main/dataset=EEG-target_model=NHiTS-online=True-attack=RMIA


<Figure size 640x480 with 0 Axes>